In [1]:
from typing import List
from rich.pretty import pprint
from phi.assistant import Assistant
from phi.tools.duckduckgo import DuckDuckGo
from phi.llm.groq import Groq
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Optional
load_dotenv()

True

## Models for Groq
### Llama 3
1. llama3-8b-8192
2. llama3-70b-8192
3. mixtral-8x7b-32768
4. gemma-7b-it

In [31]:
class SearchQuery(BaseModel):
    query: List[str] = Field(..., description="Query for researching via google search")

class Section(BaseModel):
    heading: str = Field(..., description="The heading of this section")
    content: str = Field(..., description="The content under this heading")
    sub_sections: List['Section'] = Field(default_factory=list, description="Sub-sections under this section")
    
    def to_markdown(self, level=2) -> str:
        markdown = f"{'#' * level} {self.heading}\n\n{self.content}\n\n"
        for sub_section in self.sub_sections:
            markdown += sub_section.to_markdown(level + 1)
        return markdown

class ArticleData(BaseModel):
    title: str = Field(..., description="The title of the article")
    subtitle: Optional[str] = Field(None, description="The subtitle of the article, optional")
    keywords: List[str] = Field(..., description="List of keywords for SEO purposes")
    sections: List[Section] = Field(..., description="List of main sections in the article")
    summary: Optional[str] = Field(None, description="A brief summary or conclusion of the article, optional")

    def to_markdown(self) -> str:
        markdown = f"# {self.title}\n\n"
        if self.subtitle:
            markdown += f"## {self.subtitle}\n\n"
        if self.keywords:
            markdown += f"**Keywords:** {', '.join(self.keywords)}\n\n"
        for section in self.sections:
            markdown += section.to_markdown()
        if self.summary:
            markdown += f"## Summary\n\n{self.summary}\n\n"
        return markdown


query=['site:duckduckgo.com million dollar HDB singapore', 'HDB resale prices singapore 2024', ' singapore HDB million dollar flat locations', 'HDB singapore prices 2023 vs 2024', 'latest singapore hdb flat prices may 2024']


In [32]:
assistant = Assistant(
    llm=Groq(model="llama3-70b-8192"),
    description=" You are a researcher and adapt in searching DuckDuckGo for latest trends. For a given query return the max 5 searches you would do on duckduckgo to understand more on latest news and trends. Today's date is 7th May 2024, Have a heavy bias on lates news and trends till last month.",
    #tools=[DuckDuckGo()],
    show_tool_calls=False,
    output_model=SearchQuery,
)
#assistant.print_response("1 million dollar plus HDB in singapore? Include the latest data as well.", markdown=True, stream=False)
search_terms = assistant.run("1 million dollar plus HDB in singapore? Include the latest data as well.",stream=False)
print(search_terms)

assistant_searcher = Assistant(
    llm=Groq(model="llama3-70b-8192"),
    description=" You are a famous real estate journalist & though leader working with PropertyGuru which is a real estate marketplace like zillow. Where possible give citations.",
    tools=[DuckDuckGo()],
    show_tool_calls=False,
)
context=""
for query in search_terms.query:
    context += assistant_searcher.run(query,stream=False)
print(context)

assistant_writer = Assistant(
    llm=Groq(model="llama3-70b-8192"),
    description=" You are a famous real estate journalist & though leader working with PropertyGuru which is a real estate marketplace like zillow. ",
    output_model=ArticleData
)
article = assistant_writer.run(f"Write an article on million dollar plus HDB in singapore. \n ###Additional Context: \n {context}")
print(article.to_markdown())

It seems that there are several locations in Singapore where HDB flats have been sold for over $1 million. According to the search results, some of these locations include Bukit Batok Central and Bukit Batok Street 25, as well as other areas where larger HDB flats, such as maisonettes, have been sold for high prices.

According to a article by DollarsAndSense, in 2023, 467 HDB flats across 22 estates transacted at $1 million or higher. This number has been increasing over the years, with 370 in 2022 and 259 in 2021.

It's worth noting that the prices of HDB flats can vary greatly depending on factors such as location, size, and amenities. If you're looking for more information on million-dollar HDB flats in Singapore, I recommend checking out the articles from the search results for more details.Based on the results, it appears that HDB resale prices in Singapore are stabilizing, with a 1.8% increase in Q1 2024. According to HDB's flash estimate, the resale market is showing signs of s